<a href="https://colab.research.google.com/github/inigmat/exupery/blob/main/cplex_schedule_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyP6XER

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 7.0 MB/s eta 0:00:00


In [2]:
import sys

try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        !pip install docplex
    else:
        !pip install --user docplex

try:
    import cplex
except:
    if hasattr(sys, 'real_prefix'):
        !pip install cplex
    else:
        !pip install --user cplex
        exit()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.5/646.5 kB 31.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for docplex: filename=docplex-2.30.251-py3-none-any.whl size=685954 sha256=207965866b11b3183e33ae84afecbb625fdb77b186e2cbef7493c7049961f86c
  Stored in directory: /root/.cache/pip/wheels/74/6a/00/0c08d9a7ffaa10ffd4462512d15d05197ac44d88436910bc4c
Successfully built docplex
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 20.1 MB/s eta 0:00:00


ChatGPT

In [1]:
import pandas as pd
from docplex.mp.model import Model
from xerparser.reader import Reader


# ==================================
# ФУНКЦИЯ 1: Извлечение данных из XER
# ==================================

def extract_data_from_xer(xer_path):
    xer = Reader(xer_path)
    project = xer.projects[0]

    print(f"📁 Загрузка проекта: {project.proj_short_name}")

    # 1. Задачи
    tasks = {
        a.task_id: max(1, int(a.duration or 1))  # продолжительность в днях
        for a in project.activities
    }

    # 2. Предшественники
    precedences = []
    for activity in project.activities:
        for pred in activity.predecessors:
            if pred.pred_task_id in tasks and activity.task_id in tasks:
                precedences.append((pred.pred_task_id, activity.task_id))

    # 3. Ресурсы: task_id -> (contractor_name, personnel_count)
    resources_by_id = {r.rsrc_id: r for r in xer.resources}
    resource_mapping = {}
    for ar in xer.activityresources:
        task_id = ar.task_id
        rsrc_id = ar.rsrc_id
        quantity_hr = ar.target_qty or 0

        if task_id not in tasks or rsrc_id not in resources_by_id:
            continue

        contractor = resources_by_id[rsrc_id].rsrc_name
        duration_days = tasks[task_id]
        personnel = max(1, int(quantity_hr / (8 * duration_days)))  # человек на день
        resource_mapping[task_id] = (contractor, personnel)

    # 4. Подрядчики и лимиты — можно задать руками или вытащить по max
    contractor_limits = {}
    for r in xer.resources:
        contractor_limits[r.rsrc_name] = 99  # заглушка

    # 5. Общий лимит (задать вручную или вычислить)
    total_resource_limit = 10

    return tasks, precedences, resource_mapping, contractor_limits, total_resource_limit


# ==================================
# ФУНКЦИЯ 2: Оптимизация графика (CPLEX)
# ==================================

def schedule_project(tasks, precedences, resources, contractor_limits, total_resource_limit):
    mdl = Model("Project_Scheduling_Minimize_Makespan")

    task_ids = list(tasks.keys())
    start_times = mdl.continuous_var_dict(task_ids, name="start")
    end_times = {t: start_times[t] + tasks[t] for t in task_ids}
    makespan = mdl.continuous_var(name="makespan")

    mdl.add_constraints(end_times[t] <= makespan for t in task_ids)

    for pred, succ in precedences:
        mdl.add_constraint(start_times[succ] >= end_times[pred])

    HORIZON = sum(tasks.values()) + 1
    time_points = list(range(HORIZON))
    is_active = {(t, τ): mdl.binary_var(name=f"act_{t}_{τ}") for t in task_ids for τ in time_points}

    for t in task_ids:
        for τ in time_points:
            mdl.add_indicator(is_active[t, τ], start_times[t] <= τ)
            mdl.add_indicator(is_active[t, τ], τ < end_times[t])

    for τ in time_points:
        total_usage = mdl.sum(is_active[t, τ] * resources[t][1] for t in task_ids if t in resources)
        mdl.add_constraint(total_usage <= total_resource_limit)

    mdl.minimize(makespan)
    solution = mdl.solve(log_output=True)

    if not solution:
        print("❌ Решение не найдено.")
        return None

    schedule = {
        t: {
            "start": start_times[t].solution_value,
            "end": end_times[t].solution_value,
            "contractor": resources[t][0],
            "personnel": resources[t][1]
        }
        for t in task_ids if t in resources
    }

    print(f"\n✅ Проект завершён за: {makespan.solution_value:.2f} дней")
    return schedule


# ==================================
# ФУНКЦИЯ 3: Экспорт в Excel/CSV
# ==================================

def export_schedule_to_file(schedule, filename_base="project_schedule"):
    df = pd.DataFrame([
        {
            "Task ID": task_id,
            "Contractor": info["contractor"],
            "Personnel": info["personnel"],
            "Start": round(info["start"], 2),
            "End": round(info["end"], 2),
            "Duration": round(info["end"] - info["start"], 2)
        }
        for task_id, info in schedule.items()
    ])
    df.to_csv(f"{filename_base}.csv", index=False)
    df.to_excel(f"{filename_base}.xlsx", index=False)
    print(f"\n📁 Расписание сохранено в:\n - {filename_base}.csv\n - {filename_base}.xlsx")


# ==================================
# ТОЧКА ВХОДА
# ==================================

if __name__ == "__main__":
    xer_file_path = "MDL4D.xer"  # замените на ваш путь к .xer файлу

    print("🔄 Импорт данных из XER...")
    tasks, precedences, resources, contractor_limits, total_resource_limit = extract_data_from_xer(xer_file_path)

    print("⚙️ Оптимизация графика проекта...")
    schedule = schedule_project(tasks, precedences, resources, contractor_limits, total_resource_limit)

    if schedule:
        export_schedule_to_file(schedule, filename_base="schedule_from_xer")


🔄 Импорт данных из XER...


TypeError: 'Projects' object is not subscriptable

Gemini

In [2]:
import pandas as pd
from docplex.cp.model import CpoModel
from xerparser.reader import Reader  # Используем xerparser, как в примере

def load_data_from_xer(xer_file_path: str, contractor_capacities: dict):
    """
    Извлекает данные о задачах, зависимостях и ресурсах из .xer файла,
    используя библиотеку xerparser.

    Args:
        xer_file_path (str): Путь к .xer файлу.
        contractor_capacities (dict): Словарь с максимальной численностью
                                      персонала для каждого подрядчика.
                                      {'Имя ресурса': ёмкость}.

    Returns:
        tuple: Кортеж с данными:
               (tasks_data, precedences, task_assignments)
    """
    print(f"Загрузка данных из файла: {xer_file_path}")
    xer = Reader(xer_file_path)

    # Для простоты работаем с первым проектом в файле
    if not xer.projects:
        raise ValueError("В XER файле не найдено проектов.")
    project = xer.projects[0]
    print(f"Анализ проекта: '{project.proj_short_name}'")

    # Словари для хранения извлеченных данных
    tasks_data = {}
    precedences = []
    task_assignments = {}

    # 1. Создаем карты ID в имена для удобства
    task_id_map = {t.task_id: t.task_name for t in project.activities}
    resource_id_map = {r.rsrc_id: r.rsrc_name for r in xer.resources}

    # 2. Извлекаем задачи и их длительность
    for task in project.activities:
        # Используем orig_duration, так как это плановая длительность
        tasks_data[task.task_name] = task.orig_duration or 0

    # 3. Извлекаем зависимости (предшественники)
    for pred_relation in xer.taskpreds:
        # Убеждаемся, что обе задачи из связи существуют
        if pred_relation.task_id in task_id_map and pred_relation.pred_task_id in task_id_map:
            pred_name = task_id_map[pred_relation.pred_task_id]
            succ_name = task_id_map[pred_relation.task_id]
            precedences.append((pred_name, succ_name))

    # 4. Извлекаем назначения ресурсов на задачи
    for assignment in xer.activityresources:
        if assignment.task_id in task_id_map and assignment.rsrc_id in resource_id_map:
            task_name = task_id_map[assignment.task_id]
            resource_name = resource_id_map[assignment.rsrc_id]

            # target_qty - это плановое количество единиц ресурса
            personnel_needed = int(round(assignment.target_qty or 0))

            if personnel_needed > 0:
                # Проверяем, что для этого ресурса задана ёмкость
                if resource_name not in contractor_capacities:
                    print(f"⚠️ Предупреждение: Ресурс '{resource_name}' назначен на задачу, но его ёмкость не определена в словаре 'CONTRACTOR_CAPACITIES'. Он будет проигнорирован.")
                    continue
                task_assignments[task_name] = (resource_name, personnel_needed)

    print("Данные успешно загружены.")
    return tasks_data, precedences, task_assignments


# --- 🚀 Основной блок выполнения ---
if __name__ == "__main__":
    # --- 1. Входные параметры ---
    # ⬇️ УКАЖИТЕ ПУТЬ К ВАШЕМУ .XER ФАЙЛУ
    XER_FILE_PATH = 'MDL4D.xer'

    # ⬇️ УКАЖИТЕ МАКСИМАЛЬНУЮ ЧИСЛЕННОСТЬ ДЛЯ КАЖДОГО ПОДРЯДЧИКА (РЕСУРСА)
    # Формат: {'Имя ресурса из P6': количество_сотрудников}
    CONTRACTOR_CAPACITIES = {
        'Аналитики': 2,
        'Разработчики': 5,
        'Тестировщики': 3
    }

    # ⬇️ УКАЖИТЕ ОБЩИЙ ЛИМИТ ПЕРСОНАЛА ПО ВСЕМ ПОДРЯДЧИКАМ
    TOTAL_PERSONNEL_LIMIT = 8

    try:
        tasks_data, precedences, task_assignments = load_data_from_xer(XER_FILE_PATH, CONTRACTOR_CAPACITIES)
    except FileNotFoundError:
        print(f"❌ Ошибка: Файл {XER_FILE_PATH} не найден.")
        exit()
    except Exception as e:
        print(f"❌ Произошла ошибка при чтении XER файла: {e}")
        exit()

    # --- 2. Создание модели CPLEX ---
    mdl = CpoModel(name='ProjectMinimization_from_XER')

    # --- 3. Создание переменных решения (интервалы задач) ---
    tasks = {
        task: mdl.interval_var(length=duration, name=task.replace(' ', '_')) # Имена без пробелов
        for task, duration in tasks_data.items()
    }

    # --- 4. Добавление ограничений ---
    print("Добавление ограничений в модель...")

    # Ограничения предшествования
    for pred, succ in precedences:
        if pred in tasks and succ in tasks:
             mdl.add(mdl.end_before_start(tasks[pred], tasks[succ]))

    # Ограничения по ресурсам подрядчиков и общий лимит
    total_personnel_usage = mdl.cumulative_function()

    for contractor, capacity in CONTRACTOR_CAPACITIES.items():
        contractor_usage = mdl.cumulative_function()
        for task_name, (assigned_contractor, personnel_needed) in task_assignments.items():
            if assigned_contractor == contractor and task_name in tasks:
                contractor_usage += mdl.pulse(tasks[task_name], personnel_needed)
                total_personnel_usage += mdl.pulse(tasks[task_name], personnel_needed)

        mdl.add(mdl.always_in(contractor_usage, (0, mdl.infinity()), 0, capacity))

    mdl.add(mdl.always_in(total_personnel_usage, (0, mdl.infinity()), 0, TOTAL_PERSONNEL_LIMIT))

    # --- 5. Определение целевой функции ---
    makespan = mdl.max(mdl.end_of(task) for task in tasks.values())
    mdl.add(mdl.minimize(makespan))

    # --- 6. Решение модели ---
    print("🤖 Решение модели...")
    msol = mdl.solve(TimeLimit=60, LogVerbosity='Terse')

    # --- 7. Вывод результатов ---
    if msol:
        print(f"\n✅ Оптимальное расписание найдено! Минимальный срок: {msol.get_objective_values()[0]} дней.")
        results = []
        for task_name, task_var in tasks.items():
            sol = msol.get_var_solution(task_var)
            results.append({
                "Задача": task_name,
                "Начало": sol.get_start(),
                "Окончание": sol.get_end(),
                "Длительность": sol.get_length(),
            })

        results_df = pd.DataFrame(results).sort_values(by="Начало").reset_index(drop=True)
        print("\n📅 План-график работ:")
        print(results_df.to_string())
    else:
        print("\n❌ Решение не найдено в течение установленного лимита времени.")

Загрузка данных из файла: MDL4D.xer
❌ Произошла ошибка при чтении XER файла: 'Projects' object is not subscriptable


NameError: name 'tasks_data' is not defined

DeepSeek

In [26]:
import docplex.mp.model as cpx
from xerparser.reader import Reader
import pandas as pd
from datetime import datetime, timedelta

def extract_data_from_xer(xer_file_path):
    """Извлечение данных из XER файла для оптимизации"""
    xer = Reader(xer_file_path)

    # Базовые данные
    tasks = []
    durations = {}
    predecessors = {}
    successors = {}

    # Ресурсные данные
    contractors = set()
    task_contractors = {}
    manpower = {}

    # Извлекаем задачи и их зависимости
    for activity in xer.activities:
        task_id = activity.task_id
        tasks.append(task_id)
        durations[task_id] = activity.duration or 0

        # Предшественники и последователи
        preds = [p.pred_task_id for p in activity.predecessors]
        predecessors[task_id] = preds
        for pred in preds:
            successors.setdefault(pred, []).append(task_id)

    # Извлекаем информацию о ресурсах
    for resource in xer.resources:
        contractors.add(resource.rsrc_short_name)

    for assignment in xer.activityresources:
        task_id = assignment.task_id
        contractor = assignment.rsrc_id  # Здесь нужно сопоставить ID с именем подрядчика
        qty = assignment.target_qty or 0

        if task_id in durations:
            task_contractors.setdefault(task_id, set()).add(contractor)
            manpower[(task_id, contractor)] = qty

    # Преобразуем подрядчиков в список
    contractors = list(contractors)

    # Преобразуем множества в списки для task_contractors
    for task in task_contractors:
        task_contractors[task] = list(task_contractors[task])

    return {
        'tasks': tasks,
        'durations': durations,
        'predecessors': predecessors,
        'successors': successors,
        'contractors': contractors,
        'task_contractors': task_contractors,
        'manpower': manpower
    }

def optimize_project_schedule(xer_data, total_manpower_limit):
    """Оптимизация расписания проекта на основе извлеченных данных"""
    model = cpx.Model(name="Project_Scheduling_Optimization")

    # Параметры
    tasks = xer_data['tasks']
    durations = xer_data['durations']
    predecessors = xer_data['predecessors']
    contractors = xer_data['contractors']
    task_contractors = xer_data['task_contractors']
    manpower = xer_data['manpower']

    horizon = sum(durations.values())  # Верхняя граница времени

    # Переменные
    start_time = {task: model.continuous_var(name=f"start_{task}", lb=0) for task in tasks}
    end_time = {task: model.continuous_var(name=f"end_{task}", lb=0) for task in tasks}
    project_end = model.continuous_var(name="project_end", lb=0)
    assign = {(task, c): model.binary_var(name=f"assign_{task}_{c}")
              for task in tasks for c in task_contractors.get(task, [])}

    # Ограничения
    # 1. Связь между временем начала и завершения
    for task in tasks:
        model.add_constraint(end_time[task] == start_time[task] + durations[task])

    # 2. Определение времени завершения проекта
    model.add_constraint(project_end == model.max(end_time[task] for task in tasks))

    # 3. Ограничения предшествования
    for task in tasks:
        for pred in predecessors.get(task, []):
            if pred in tasks:
                model.add_constraint(start_time[task] >= end_time[pred])

    # 4. Каждая задача назначается ровно одному подрядчику
    for task in tasks:
        if task in task_contractors:
            model.add_constraint(
                model.sum(assign[task, c] for c in task_contractors[task]) == 1
            )

    # 5. Ограничение на общую численность персонала
    total_manpower = model.sum(
        manpower.get((task, c), 0) * assign[task, c] * durations[task]
        for task in tasks
        for c in task_contractors.get(task, [])
    )
    model.add_constraint(total_manpower <= total_manpower_limit * horizon)

    # Целевая функция
    model.minimize(project_end)

    # Решение
    solution = model.solve()

    if solution:
        # Формируем результаты
        results = {
            'project_end': solution.get_value(project_end),
            'schedule': {},
            'assignments': {}
        }

        for task in tasks:
            results['schedule'][task] = {
                'start': solution.get_value(start_time[task]),
                'end': solution.get_value(end_time[task]),
                'contractor': next(
                    (c for c in task_contractors.get(task, [])
                     if solution.get_value(assign[task, c]) > 0.9), None)
            }

        return model, solution, results
    else:
        print("Решение не найдено")
        return None, None, None

def save_results_to_xer(xer_file_path, results, output_file_path):
    """Сохранение результатов оптимизации обратно в XER файл"""
    xer = Reader(xer_file_path)

    for activity in xer.activities:
        task_id = activity.task_id
        if task_id in results['schedule']:
            schedule = results['schedule'][task_id]
            # Здесь должна быть логика обновления дат в активности
            # activity.act_start_date = ...
            # activity.act_end_date = ...

    xer.write(output_file_path)

def export_to_excel(results, excel_file_path, project_start_date=None):
    """Экспорт результатов оптимизации в Excel файл"""
    data = []
    for task, schedule in results['schedule'].items():
        start_date = project_start_date + timedelta(days=schedule['start']) if project_start_date else schedule['start']
        end_date = project_start_date + timedelta(days=schedule['end']) if project_start_date else schedule['end']

        data.append({
            'Task': task,
            'Start': start_date,
            'End': end_date,
            'Duration': schedule['end'] - schedule['start'],
            'Contractor': schedule['contractor'],
            'Start Day': schedule['start'],
            'End Day': schedule['end']
        })

    df = pd.DataFrame(data).sort_values('Start Day')

    with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Schedule', index=False)

        summary_data = {
            'Metric': ['Project Duration (days)', 'Total Manpower Limit'],
            'Value': [results['project_end'], total_manpower_limit]
        }
        pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)

        workbook = writer.book
        worksheet = writer.sheets['Schedule']

        worksheet.column_dimensions['A'].width = 15
        worksheet.column_dimensions['B'].width = 20
        worksheet.column_dimensions['C'].width = 20
        worksheet.column_dimensions['D'].width = 15
        worksheet.column_dimensions['E'].width = 20

        date_format = 'dd-mm-yyyy'
        for cell in worksheet['B'][1:]:
            cell.number_format = date_format
        for cell in worksheet['C'][1:]:
            cell.number_format = date_format

def export_to_csv(results, csv_file_path):
    """Экспорт результатов оптимизации в CSV файл"""
    data = []
    for task, schedule in results['schedule'].items():
        data.append({
            'Task': task,
            'Start': schedule['start'],
            'End': schedule['end'],
            'Duration': schedule['end'] - schedule['start'],
            'Contractor': schedule['contractor']
        })

    pd.DataFrame(data).sort_values('Start').to_csv(csv_file_path, index=False)

if __name__ == "__main__":
    # Конфигурация
    xer_file_path = "MDL4D.xer"
    output_xer_path = "optimized_project.xer"
    excel_output_path = "optimized_schedule.xlsx"
    csv_output_path = "optimized_schedule.csv"
    total_manpower_limit = 240
    project_start_date = datetime.now()  # Дата начала проекта

    print("Загрузка данных из XER файла...")
    xer_data = extract_data_from_xer(xer_file_path)

    print("Оптимизация расписания проекта...")
    model, solution, results = optimize_project_schedule(xer_data, total_manpower_limit)

    if results:
        print(f"\nОптимальное время завершения проекта: {results['project_end']} дней")

        print("\nСохранение результатов...")
        save_results_to_xer(xer_file_path, results, output_xer_path)
        export_to_excel(results, excel_output_path, project_start_date)
        export_to_csv(results, csv_output_path)

        print("\nГотово! Результаты сохранены в:")
        print(f"- XER файл: {output_xer_path}")
        print(f"- Excel файл: {excel_output_path}")
        print(f"- CSV файл: {csv_output_path}")
    else:
        print("Не удалось найти решение для оптимизации расписания")

Загрузка данных из XER файла...
Оптимизация расписания проекта...

Оптимальное время завершения проекта: 90.0 дней

Сохранение результатов...

Готово! Результаты сохранены в:
- XER файл: optimized_project.xer
- Excel файл: optimized_schedule.xlsx
- CSV файл: optimized_schedule.csv


In [8]:
xer_data.keys()

dict_keys(['tasks', 'durations', 'predecessors', 'successors', 'contractors', 'task_contractors', 'manpower'])

In [22]:
xer_data.get('contractors')

['MDL4E', 'W-2', 'WE', 'W-1', 'W-3', 'MDL4W']

In [24]:
total_manpower_limit

3